In [1]:
import sys
import seaborn as sns
import pandas as pd 
import numpy as np
from scipy.spatial.distance import squareform, pdist
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
import torch
import anndata as an
import scanpy as sc
import os
from importlib import reload

from datasets import Dataset, load_from_disk
from datasets import load_dataset
#YOU are the problem >:(
from geneformer import EmbExtractor
import geneformer

# local imports
sys.path.insert(0, '../../scripts/')
import geneformer_utils as gtu

sns.set_style('white')
torch.cuda.empty_cache()


print("done")

/home/oliven/miniconda3/envs/geneformer2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


done


In [2]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

# Load the model

In [3]:
"""Load the model"""
# model_path = "/nfs/turbo/umms-indikar/shared/projects/geneformer/fine_tuned_models/geneformer-6L-30M_CellClassifier_cardiomyopathies_220224/"
# model_path = "/nfs/turbo/umms-indikar/shared/projects/geneformer/geneformer-12L-30M/"
model_path = "/scratch/indikar_root/indikar1/shared_data/geneformer/fine_tune/240715_geneformer_cellClassifier_no_induced/ksplit1/"
model = gtu.load_model(model_path)
print('loaded!')

Some weights of BertForMaskedLM were not initialized from the model checkpoint at /scratch/indikar_root/indikar1/shared_data/geneformer/fine_tune/240715_geneformer_cellClassifier_no_induced/ksplit1/ and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


loaded!


In [4]:
# model

In [5]:
token_data_path = "/scratch/indikar_root/indikar1/shared_data/geneformer/resources/token_mapping.csv"
token_df = pd.read_csv(token_data_path)
token_df.head()

,gene_id,token_id,gene_name,nonzero_median,gene_version,gene_biotype,Chromosome,Start,End,scenic_tf
0,<pad>,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,<mask>,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,ENSG00000000003,2,TSPAN6,2.001186,15.0,protein_coding,X,100627107.0,100639991.0,False
3,ENSG00000000005,3,TNMD,3.228213,6.0,protein_coding,X,100584935.0,100599885.0,False
4,ENSG00000000419,4,DPM1,2.218874,14.0,protein_coding,20,50934866.0,50959140.0,False


# Load data

In [6]:
dpath = "/scratch/indikar_root/indikar1/shared_data/geneformer/datasets/"


def load_data(path, sample_size=None):
    """
    Loads data from a file, processes cell types, and returns a DataFrame.

    Args:
        path (str): The path to the dataset file.
        sample_size (int, optional): Number of cells to sample. Defaults to None.
    Returns:
        pandas.DataFrame: The processed DataFrame.
    """

    df = gtu.load_data_as_dataframe(path, num_cells=sample_size, shuffle=True)

    if "iHSC" in path:
        df['cell_type'] = "iHSC"
    elif "pellin" in path:
        df['cell_type'] = df['dataset']
    elif "weng" in path:
        df['cell_type'] = df['STD.CellType']
    else:
        df['cell_type'] = df['free_annotation']

    # Extract basename without extension and assign to 'dataset' column
    df['dataset'] = os.path.splitext(os.path.basename(path))[0]
    df = df[['input_ids', 'cell_type', 'dataset', 'length']]

    return df

sample_size = 1000

df = []

for dataset in os.listdir(dpath):
    data_path = f"{dpath}{dataset}"
    tmp = load_data(data_path, sample_size)
    
    df.append(tmp)
    
df = pd.concat(df)
print(f"{df.shape=}")
df = df[df['cell_type'].isin(['HSC', 'Fibroblasts'])]
print(f"{df.shape=}")

df['cell_type'].value_counts()

data = Dataset.from_pandas(df)
data

df.shape=(12000, 4)
df.shape=(1639, 4)


Dataset({
    features: ['input_ids', 'cell_type', 'dataset', 'length', '__index_level_0__'],
    num_rows: 1639
})

In [ ]:
print(df['cell_type'].value_counts())

In [ ]:
reload(gtu)
torch.cuda.empty_cache()
embs = gtu.extract_embedding_in_mem(model, data, layer_to_quant=-1)
print(f"{embs.shape=}")

# translate into an anndata object and plot
adata = gtu.embedding_to_adata(embs)
adata.obs = df.copy()

sc.tl.pca(adata, n_comps=25)
sc.pp.neighbors(adata, n_neighbors=200)
sc.tl.umap(adata, 
           min_dist=0.75,
          )

plt.rcParams['figure.dpi'] = 200
plt.rcParams['figure.figsize'] = 7, 7

sc.pl.umap(
    adata,
    color=["cell_type", "dataset"],
    ncols=1,
    size=30,
)

adata

# Clean up cell types

In [ ]:
fpath = "ihsc_cell_types.csv"
cell_map = pd.read_csv(fpath, comment="#")

adata.obs = pd.merge(adata.obs, 
                     cell_map,
                     how='left',
                     left_on='cell_type',
                     right_on='label',
                    )

plt.rcParams['figure.dpi'] = 200
plt.rcParams['figure.figsize'] = 7, 7
sc.pl.umap(
    adata,
    color=["standardized_cell_type","broad_type"],
    ncols=1,
    size=30,
)

In [ ]:
df[df['dataset'] == "pellin"]['cell_type'].value_counts()

# COMPUTE HSC CENTROID

In [ ]:
HSC_embeddings = adata[adata.obs['cell_type'] == 'HSC', :].X
hsc_cenroid = HSC_embeddings.mean(axis=0)
hsc_cenroid = hsc_cenroid.reshape(1, -1)
print(f"{hsc_cenroid.shape=}")

In [ ]:
tfs = [
    'FOSB',
    'FOS',
    'REL',
    'STAT5A',
    'GATA2',
]

tf_tokens = token_df[token_df['gene_name'].isin(tfs)]['token_id'].to_list()
print(tf_tokens)

# Perturb the Data

In [ ]:
def perturb_data(df, tokens):
    """
    Perturbs the input_ids in a DataFrame by prepending a list of tokens.

    Args:
      df (pandas.DataFrame): The input DataFrame with an 'input_ids' column.
      tokens (list): The list of tokens to prepend.

    Returns:
      pandas.DataFrame: The combined DataFrame with original and perturbed data.
    """

    perturbed = df.copy()
    perturbed['state'] = 'perturbed'
    perturbed['input_ids'] = perturbed['input_ids'].apply(
      lambda x: list(tokens) + list(x)[:len(x) - len(tokens)]
    )

    df['state'] = 'raw'
    combined_df = pd.concat([df, perturbed])
    return combined_df

sample_size = 400
fb_cells = adata.obs[adata.obs['cell_type'] == 'Fibroblasts'].copy()
fb_cells = fb_cells.sample(sample_size)
fb_cells = fb_cells.reset_index(names='cell_id')

perturbed = perturb_data(fb_cells, tf_tokens)
perturbed = perturbed.reset_index(drop=True)
perturbed_data = Dataset.from_pandas(perturbed)

# extract the embeddings
torch.cuda.empty_cache()
test_embs = gtu.extract_embedding_in_mem(model, perturbed_data, layer_to_quant=-1)

perturbed

# Compute Distance to HSC

In [ ]:
metric = 'cosine'
Xa = test_embs.to_numpy()
Xb = hsc_cenroid

print(f"{Xa.shape=} {Xb.shape=}")
D = cdist(Xa, Xb, metric=metric)
print(f"{D.shape=}")

perturbed['distance_to_hsc'] = D.ravel()
perturbed

# Compute distance between raw and perturbed states

In [ ]:
P = squareform(pdist(Xa, metric=metric))
print(f"{P.shape=}")
print(P)

In [ ]:
result = []

for cell_id, group in perturbed.groupby('cell_id'):
    idx = group.index.to_list()
    
    raw_row = group.iloc[0]
    perturbed_row = group.iloc[1]

    row = {
        'cell_id' : cell_id,
        'raw_to_hsc' : raw_row['distance_to_hsc'],
        'perturbed_to_hsc' : perturbed_row['distance_to_hsc'],
        'raw_to_perturbed' : P[idx[0], idx[1]],
    }
    
    result.append(row)
    
result = pd.DataFrame(result)
result.head()

In [ ]:
plt.rcParams['figure.dpi'] = 75
sns.histplot(data=result, 
             x='raw_to_hsc')

In [ ]:
plt.rcParams['figure.dpi'] = 75
sns.histplot(data=result, 
             x='perturbed_to_hsc')

In [ ]:
plt.rcParams['figure.dpi'] = 75
sns.histplot(data=result, 
             x='raw_to_perturbed')

In [ ]:
break